22BAI10175
Ashrita Vinod
Tutorial-1

In [6]:
import kagglehub


path = kagglehub.dataset_download("fanconic/skin-cancer-malignant-vs-benign")

print("Path to dataset files:", path)

100%|██████████| 325M/325M [00:04<00:00, 78.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs-benign/versions/4


In [7]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs-benign/versions/4'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Streaming output truncated to the last 5000 lines.
/root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs-benign/versions/4/data/train/benign/811.jpg
/root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs-benign/versions/4/data/train/benign/1630.jpg
/root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs-benign/versions/4/data/train/benign/1535.jpg
/root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs-benign/versions/4/data/train/benign/399.jpg
/root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs-benign/versions/4/data/train/benign/555.jpg
/root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs-benign/versions/4/data/train/benign/1298.jpg
/root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs-benign/versions/4/data/train/benign/1165.jpg
/root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs-benign/versions/4/data/train/benign/764.jpg
/root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs

In [4]:
import tensorflow as tf
import numpy as np
import pathlib
import datetime

# printout versions
print(f"Tensor Flow Version: {tf.__version__}")
print(f"numpy Version: {np.version.version}")

Tensor Flow Version: 2.17.1
numpy Version: 1.26.4


In [10]:
data_dir = pathlib.Path("/root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs-benign/versions/4/train")
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)
# classnames in the dataset specified
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt" ])
print(CLASS_NAMES)
# print length of class names
output_class_units = len(CLASS_NAMES)
print(output_class_units)

2637
['benign' 'malignant']
2


In [11]:
model = tf.keras.models.Sequential([
    # 1st conv
  tf.keras.layers.Conv2D(96, (11,11),strides=(4,4), activation='relu', input_shape=(227, 227, 3)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, strides=(2,2)),
    # 2nd conv
  tf.keras.layers.Conv2D(256, (11,11),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
     # 3rd conv
  tf.keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
    # 4th conv
  tf.keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
    # 5th Conv
  tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, strides=(2, 2)),
  # To Flatten layer
  tf.keras.layers.Flatten(),
  # To FC layer 1
  tf.keras.layers.Dense(4096, activation='relu'),
    # add dropout 0.5 ==> tf.keras.layers.Dropout(0.5),
  #To FC layer 2
  tf.keras.layers.Dense(4096, activation='relu'),
    # add dropout 0.5 ==> tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(output_class_units, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
data_dir2 = pathlib.Path("/root/.cache/kagglehub/datasets/fanconic/skin-cancer-malignant-vs-benign/versions/4/test")

BATCH_SIZE = 32
IMG_HEIGHT = 227
IMG_WIDTH = 227
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)


image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES))

val_data_gen = image_generator.flow_from_directory(directory=str(data_dir2),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES))

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [14]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [15]:
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.SensitivityAtSpecificity(0.5), tf.keras.metrics.SpecificityAtSensitivity(0.5), tf.keras.metrics.AUC(curve='ROC')])


model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 55, 55, 96)          │          34,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 55, 55, 96)          │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 27, 27, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 27, 27, 256)         │       2,973,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 27, 27, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 27, 27, 384)         │         885,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 27, 27, 384)         │           1,536 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 27, 27, 384)         │       1,327,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 27, 27, 384)         │           1,536 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 27, 27, 256)         │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 27, 27, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 13, 13, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 43264)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4096)                │     177,213,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │           8,194 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 200,114,946 (763.38 MB)

 Trainable params: 200,112,194 (763.37 MB)

 Non-trainable params: 2,752 (10.75 KB)

In [17]:

STEPS_PER_EPOCH = int(np.ceil(image_count/BATCH_SIZE))


history = model.fit(
      train_data_gen,
      steps_per_epoch=STEPS_PER_EPOCH,
      epochs=50,
      validation_data=val_data_gen
)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


83/83 ━━━━━━━━━━━━━━━━━━━━ 74s 580ms/step - accuracy: 0.5919 - auc: 0.5995 - loss: 26.7675 - precision: 0.5546 - recall: 0.7176 - sensitivity_at_specificity: 0.6594 - specificity_at_sensitivity: 0.6520 - val_accuracy: 0.4606 - val_auc: 0.4530 - val_loss: 5.4417 - val_precision: 0.4982 - val_recall: 0.8591 - val_sensitivity_at_specificity: 0.3924 - val_specificity_at_sensitivity: 0.4409
Epoch 2/50


/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.0000e+00 - auc: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - sensitivity_at_specificity: 0.0000e+00 - specificity_at_sensitivity: 0.0000e+00 - val_accuracy: 0.4606 - val_auc: 0.4530 - val_loss: 5.4417 - val_precision: 0.4982 - val_recall: 0.8591 - val_sensitivity_at_specificity: 0.3924 - val_specificity_at_sensitivity: 0.4409
Epoch 3/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 20s 233ms/step - accuracy: 0.7312 - auc: 0.8160 - loss: 0.5124 - precision: 0.6709 - recall: 0.8687 - sensitivity_at_specificity: 0.9085 - specificity_at_sensitivity: 0.8958 - val_accuracy: 0.7212 - val_auc: 0.8033 - val_loss: 0.5206 - val_precision: 0.6552 - val_recall: 0.8667 - val_sensitivity_at_specificity: 0.8909 - val_specificity_at_sensitivity: 0.8742
Epoch 4/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.0000e+00 - auc: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - sensitivity_at_specificity: 0

In [19]:
model.save('/root/.cache/kagglehub/model.h5')
print("Total time: ", time.time() - start, "seconds")

Total time:  1093.0489897727966 seconds


In [22]:

start_eval = time.time()

results = model.evaluate(val_data_gen, steps=STEPS_PER_EPOCH, verbose=1)
end_eval = time.time()


loss = results[0]
accuracy = results[1]

# Print evaluation metrics and time taken
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.8086 - auc: 0.8961 - loss: 0.4898 - precision: 0.7818 - recall: 0.8532 - sensitivity_at_specificity: 0.9708 - specificity_at_sensitivity: 0.9717 
Validation Loss: 0.4902
Validation Accuracy: 0.8076


/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
